In [10]:
# Import modules
import pandas as pd
import gensim
from gensim.corpora import Dictionary
from gensim import corpora
from gensim.models.ldamodel import LdaModel
import scipy.sparse as sp
from scipy.sparse import csr_matrix
import numpy as np

import pyLDAvis
import pyLDAvis.gensim_models

In [11]:
# 讀入document-term dataframe
res = pd.read_csv("/Users/deankuo/Desktop/python/dissertation_replicate/Catalinac_TDM_new.csv", encoding="UTF-8")
# remove the first column 'Term' from the data frame
res2 = res.iloc[:, 1:].values

# set row names as the values of the first column
row_names = res.iloc[:, 0].values
res2 = pd.DataFrame(res2, index=row_names) # type: ignore
res2 = res2.transpose()
res2

,政治,日本,社会,実現,改革,国民,教育,企業,年金,ひと,...,都会,度目,党員,灯,難問,爆発,本質,無法,夜明け,郎
0,1,1,1,0,2,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,2,3,0,0,4,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,16,0,3,2,0,4,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,0,2,4,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,0,2,0,5,1,2,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7492,3,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7493,0,3,0,0,1,0,7,1,3,1,...,0,0,0,0,0,0,0,0,0,0
7494,0,1,1,2,0,1,1,0,3,2,...,0,0,0,0,0,0,0,0,0,0
7495,0,0,0,2,2,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
def create_bow_from_sparse_df(df):
    bow = []
    for i in range(df.shape[0]):
        doc_words = []
        for j, val in zip(df.columns, df.iloc[i]):
            if val > 0:
                doc_words += [j] * val
        bow.append(doc_words)
    return bow

df_bow = create_bow_from_sparse_df(res2)
df_bow[0][:20] # 1986 愛知県第１区 田辺広雄 Manifesto

['政治',
 '日本',
 '社会',
 '改革',
 '改革',
 '企業',
 '企業',
 '経済',
 '経済',
 '目指す',
 '地域',
 '充実',
 '平和',
 '支援',
 '安心',
 '中小',
 '中小',
 '政策',
 '作り',
 '子供']

In [14]:
id2word = corpora.Dictionary(df_bow)

corpus = []
for text in df_bow:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])
word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 3), (13, 1), (14, 1), (15, 2), (16, 1), (17, 6), (18, 1), (19, 1)]
お世話


In [15]:
# 訓練LDA模型
num_topics = 69
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics,
                                            random_state=42)

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
38     0.074138  0.204708       1        1  9.998959
45     0.079513 -0.025195       2        1  9.068241
58     0.007207  0.040772       3        1  7.478076
67    -0.004202 -0.131816       4        1  7.339696
33     0.125710  0.162960       5        1  5.682491
...         ...       ...     ...      ...       ...
49    -0.015619  0.068499      65        1  0.008358
27    -0.061429  0.057884      66        1  0.008358
43    -0.000331  0.056709      67        1  0.008358
19    -0.022695  0.067682      68        1  0.008358
0     -0.038874  0.002549      69        1  0.008358

[69 rows x 5 columns], topic_info=    Term          Freq         Total Category  logprob  loglift
64    政治  26859.000000  26859.000000  Default  30.0000  30.0000
68    日本  21119.000000  21119.000000  Default  29.0000  29.0000
86    社会  17847.000000  17847.000000  Default  28.0000  28.0000
63    改革  15656.000000  15656.000000  Default  27.0000  27.0000
221   国民  14211.000000  14211.000000  Default  26.0000  26.0000
..   ...           ...           ...      ...      ...      ...
166   消費      0.444261   7519.076118  Topic69  -5.5196  -0.3468
18    作り      0.425997   5833.640297  Topic69  -5.5616  -0.1350
325   生活      0.419987   6133.089204  Topic69  -5.5758  -0.1992
65    政策      0.410672   5851.465580  Topic69  -5.5982  -0.1746
210    力      0.407051   5647.973342  Topic69  -5.6071  -0.1481

[5213 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2362      1  0.073750   JR
2362      2  0.090139   JR
2362      3  0.032778   JR
2362      4  0.024583   JR
2362      5  0.016389   JR
...     ...       ...  ...
1507     21  0.015623   黒字
1507     22  0.210913   黒字
1507     26  0.007812   黒字
1507     30  0.234348   黒字
1507     39  0.007812   黒字

[38423 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[39, 46, 59, 68, 34, 10, 3, 67, 18, 15, 54, 69, 30, 8, 2, 42, 24, 17, 4, 37, 16, 6, 40, 58, 33, 35, 43, 23, 62, 12, 47, 36, 56, 55, 57, 51, 29, 13, 53, 52, 65, 41, 22, 45, 60, 31, 21, 32, 11, 61, 64, 49, 5, 9, 25, 26, 14, 66, 63, 19, 38, 7, 27, 48, 50, 28, 44, 20, 1])

In [17]:
# 查看主題
lda_model.print_topics()

[(36,
  '0.026*"対策" + 0.019*"図る" + 0.019*"整備" + 0.015*"農業" + 0.014*"充実" + 0.013*"振興" + 0.011*"安定" + 0.011*"推進" + 0.010*"医療" + 0.010*"地域"'),
 (57,
  '0.031*"市民" + 0.027*"社会" + 0.022*"政治" + 0.018*"目指す" + 0.017*"子供" + 0.014*"教育" + 0.012*"安心" + 0.011*"実現" + 0.011*"福祉" + 0.010*"環境"'),
 (46,
  '0.029*"社会" + 0.023*"日本" + 0.019*"改革" + 0.017*"共生" + 0.016*"教育" + 0.015*"自然" + 0.014*"環境" + 0.014*"子供" + 0.012*"平和" + 0.011*"憲法"'),
 (48,
  '0.168*"発" + 0.014*"鹿児島" + 0.013*"政治" + 0.013*"社会" + 0.011*"教育" + 0.011*"日本" + 0.008*"主義" + 0.008*"力" + 0.007*"守る" + 0.007*"造り"'),
 (20,
  '0.233*"大阪" + 0.100*"簡易" + 0.046*"疑惑" + 0.014*"政治" + 0.010*"ひと" + 0.009*"局" + 0.008*"守る" + 0.008*"税" + 0.007*"市民" + 0.007*"委員"'),
 (17,
  '0.020*"改革" + 0.017*"社会" + 0.016*"政治" + 0.015*"日本" + 0.013*"問題" + 0.012*"経済" + 0.012*"地方" + 0.011*"政策" + 0.010*"地域" + 0.010*"環境"'),
 (47,
  '0.014*"政治" + 0.011*"税" + 0.009*"日本" + 0.009*"国民" + 0.007*"推進" + 0.007*"整備" + 0.007*"自民党" + 0.007*"政策" + 0.006*"消費" + 0.006*"廃止"'),
 (61,
  '0.037*"税" + 0

In [146]:
# 利用tf-idf来做为特征进行处理
texts_tf_idf = gensim.models.TfidfModel(corpus)[corpus]
lda = gensim.models.ldamodel.LdaModel(corpus=texts_tf_idf, id2word=id2word, num_topics=num_topics)

In [148]:
for index,topic in lda.print_topics(5):
    print(topic)

0.012*"グローバル" + 0.010*"省庁" + 0.009*"ODA" + 0.008*"両立" + 0.005*"職業" + 0.005*"委譲" + 0.005*"大量" + 0.005*"無駄" + 0.005*"説明" + 0.004*"発想"
0.015*"立て直し" + 0.015*"京" + 0.004*"本" + 0.004*"戻る" + 0.004*"育む" + 0.003*"主役" + 0.003*"減税" + 0.003*"増える" + 0.003*"昭和" + 0.003*"都市"
0.017*"元気" + 0.014*"富山" + 0.013*"戦力" + 0.010*"設計" + 0.009*"修正" + 0.009*"横浜" + 0.008*"至上" + 0.007*"本音" + 0.005*"男" + 0.005*"語る"
0.010*"成し遂げる" + 0.008*"閉塞" + 0.007*"役所" + 0.007*"責任" + 0.007*"市民" + 0.007*"倍増" + 0.006*"立ち向かう" + 0.006*"郎" + 0.006*"官僚" + 0.005*"独自"
0.038*"山梨" + 0.013*"地震" + 0.009*"像" + 0.008*"駅前" + 0.008*"離合" + 0.008*"集散" + 0.004*"共生" + 0.004*"規制" + 0.004*"官" + 0.004*"スポーツ"
